In [19]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id', 
             'air_reserved_visitors', 'hpg_reserved_visitors','reserved_visitors']

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
OutputDir = '%s/l2/kfold' % DataBaseDir
kfold = 5
TopTreePair = 50000
TopN = 10000
threshold = 20
SingleModels = ['lgb_l2', 'xgb_rmse','lgb_huber', 'lgb_fair', 'etr', 'en', 'rf',  'gbr_ls', 'gbr_huber', 'gbr_lad', 'lassolars', 'rgf']
start_time = datetime.datetime.now()

#### load pair indexs1
meta_tree_mod1 = 'meta_trees_25_500'
InputDir1 = '%s/%s/l1/kfold' % (DataBaseDir, meta_tree_mod1)
PairIndex1 = pd.read_csv('%s/%s/l1/pairs/%s_top%s_holdout.csv' % (DataBaseDir, meta_tree_mod1, meta_tree_mod1, TopTreePair))
PairIndex1.columns = ['a', 'b']
PairIndex1 = PairIndex1.loc[:TopN,]
va = PairIndex1['a'].value_counts()
vb = PairIndex1['b'].value_counts()
vc = {}
for k in va.keys():
    vc[k] = va[k]
for k in vb.keys():
    if(k not in vc):
        vc[k] = vb[k]
    else:
        vc[k] += vb[k]
TopModelIndex1 = [v for v in vc if(vc[v] > threshold)]
print('top %s models for %s.' % (len(TopModelIndex1), meta_tree_mod1))

####
meta_tree_mod2 = 'meta_trees_50_500'
InputDir2 = '%s/%s/l1/kfold' % (DataBaseDir, meta_tree_mod2)
PairIndex2 = pd.read_csv('%s/%s/l1/pairs/%s_top%s_holdout.csv' % (DataBaseDir, meta_tree_mod2, meta_tree_mod2, TopTreePair))
PairIndex2.columns = ['a', 'b']
PairIndex2 = PairIndex2.loc[:TopN,]
va = PairIndex2['a'].value_counts()
vb = PairIndex2['b'].value_counts()
vc = {}
for k in va.keys():
    vc[k] = va[k]
for k in vb.keys():
    if(k not in vc):
        vc[k] = vb[k]
    else:
        vc[k] += vb[k]
TopModelIndex2 = [v for v in vc if(vc[v] > threshold)]
print('top %s models for %s.' % (len(TopModelIndex2), meta_tree_mod2))

####
meta_tree_mod3 = 'meta_trees_75_500'
InputDir3 = '%s/%s/l1/kfold' % (DataBaseDir, meta_tree_mod3)
PairIndex3 = pd.read_csv('%s/%s/l1/pairs/%s_top%s_holdout.csv' % (DataBaseDir, meta_tree_mod3, meta_tree_mod3, TopTreePair))
PairIndex3.columns = ['a', 'b']
PairIndex3 = PairIndex3.loc[:TopN,]
va = PairIndex3['a'].value_counts()
vb = PairIndex3['b'].value_counts()
vc = {}
for k in va.keys():
    vc[k] = va[k]
for k in vb.keys():
    if(k not in vc):
        vc[k] = vb[k]
    else:
        vc[k] += vb[k]
TopModelIndex3 = [v for v in vc if(vc[v] > threshold)]
print('top %s models for %s.' % (len(TopModelIndex3), meta_tree_mod3))
# sys.exit(1)
#PairNum = len(PairIndex)
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
start = time.time()
strategy1 = 'lgb_l2_%s' % meta_tree_mod1
strategy2 = 'lgb_l2_%s' % meta_tree_mod2
strategy3 = 'lgb_l2_%s' % meta_tree_mod3

for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir1, fold)
    FoldValid = pd.DataFrame()
    FoldHoldout = pd.DataFrame()
    FoldTest = pd.DataFrame()
    for i in range(len(TopModelIndex1)):
        colname = '%s_%s' % (strategy1, TopModelIndex1[i])
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid[colname] = valid[colname]
            FoldHoldout[colname] = holdout[colname]
            FoldTest[colname] = test[colname]
    FoldInputDir = '%s/%s' % (InputDir2, fold)
    for i in range(len(TopModelIndex2)):
        colname = '%s_%s' % (strategy2, TopModelIndex2[i])
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[colname] = valid[colname]
        FoldHoldout[colname] = holdout[colname]
        FoldTest[colname] = test[colname]
    FoldInputDir = '%s/%s' % (InputDir3, fold)
    for i in range(len(TopModelIndex3)):
        colname = '%s_%s' % (strategy3, TopModelIndex3[i])
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, colname), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[colname] = valid[colname]
        FoldHoldout[colname] = holdout[colname]
        FoldTest[colname] = test[colname]
    FoldInputDir = '../../data/l1/kfold/%s' % (fold)
    for i in range(len(SingleModels)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, SingleModels[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, SingleModels[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, SingleModels[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        FoldValid[SingleModels[i]] = valid[SingleModels[i]]
        FoldHoldout[SingleModels[i]] = holdout[SingleModels[i]]
        FoldTest[SingleModels[i]] = test[SingleModels[i]]
    ###
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
    end = time.time()
    print('fold %s done, time elapsed %ss' % (fold, (end - start)))
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ TestData')
print(test_dfs[0].head(5))
print('-------------------------------------')
# sys.exit(1)
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in drop_cols]
    # train
    model = linear_model.ElasticNet(alpha= 0.0001, l1_ratio= 0.2, max_iter= 400, tol= 1e-4, selection= 'random', random_state= 2017)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputStrategy = 'top_%s_%s_%s_meta_trees' % (len(TopModelIndex1), len(TopModelIndex2), len(TopModelIndex3))
OutputFileName = '%s_submit_%s' % (OutputStrategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

top 188 models for meta_trees_25_500.
top 214 models for meta_trees_50_500.
top 204 models for meta_trees_75_500.
fold 0 done, time elapsed 108.97383618354797s
fold 1 done, time elapsed 219.16406297683716s
fold 2 done, time elapsed 328.4018130302429s
fold 3 done, time elapsed 438.42561888694763s
fold 4 done, time elapsed 549.727949142456s
----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors  lgb_l2_meta_trees_25_500_17  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197                     3.388881   
1  air_ba937bf13d40fb24 2016-01-29  3.295837                     3.345007   
2  air_ba937bf13d40fb24 2016-01-30  1.945910                     3.152395   
3  air_ba937bf13d40fb24 2016-02-10  3.496508                     3.267363   
4  air_ba937bf13d40fb24 2016-02-13  2.197225                     3.097088   

   lgb_l2_meta_trees_25_500_31  lgb_l2_meta_trees_25_500_57  \
0                     3.435833                     3.46

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.497014, holdout score 0.493867, valid length 45371
saving for 0th fold data done.
fold 1: valid score 0.488093, holdout score 0.493170, valid length 45371
saving for 1th fold data done.
fold 2: valid score 0.490437, holdout score 0.492607, valid length 45371
saving for 2th fold data done.
fold 3: valid score 0.493821, holdout score 0.492577, valid length 45370
saving for 3th fold data done.
fold 4: valid score 0.488796, holdout score 0.492406, valid length 45370
saving for 4th fold data done.
zip ../../data/l2/submit/top_188_214_204_meta_trees_submit_2018-01-19.zip ../../data/l2/submit/top_188_214_204_meta_trees_submit_2018-01-19.csv

CV score 0.4916, Holdout score 0.4929, Elapsed time: 992.00s

